## Imports


In [87]:
!pip install folium

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
#import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in


In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

Aug. 12 Aug. 18


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df.sort_values(by='PRICE',ascending=False)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
255,PAST SALE,June-24-2025,Single Family Residential,55 E San Marino Dr,Miami Beach,FL,33139.0,46000000.0,6.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/55-E-San...,MARMLS,A11756792,N,Y,25.788982,-80.162002
205,PAST SALE,July-25-2025,Single Family Residential,555 Reinante Ave,Coral Gables,FL,33156.0,40000000.0,7.0,8.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/555-Rei...,MARMLS,A11847036,N,Y,25.687573,-80.263132
229,PAST SALE,NaN,Multi-Family (5+ Unit),514 SW 22nd Ave,Miami,FL,33135.0,17300000.0,72.0,144.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Miami/514-SW-22nd-Av...,NaN,NaN,N,Y,25.767603,-80.231019
39,PAST SALE,June-30-2025,Condo/Co-op,17749 Collins Ave Unit TS2401/02,Sunny Isles Beach,FL,33160.0,13500000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11825367,N,Y,25.940065,-80.120264
153,PAST SALE,June-6-2025,Single Family Residential,2037 NE 121st Rd,North Miami,FL,33181.0,9450000.0,7.0,8.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/2037-NE-...,MARMLS,A11803708,N,Y,25.886791,-80.155977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,PAST SALE,NaN,Condo/Co-op,1659 NE Miami Gardens Dr #110,Miami,FL,33179.0,100000.0,1.0,1.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Miami/1659-NE-Miami-...,NaN,NaN,N,Y,25.946706,-80.166988
40,PAST SALE,NaN,Condo/Co-op,15325 SW 106th Ter #626,Miami,FL,33196.0,100000.0,1.0,1.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Miami/15325-SW-106th...,NaN,NaN,N,Y,25.667628,-80.444757
111,PAST SALE,June-20-2025,Single Family Residential,5296 NW 4th St,Miami,FL,33126.0,75000.0,4.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/5296-NW-4th-St...,MARMLS,A11816704,N,Y,25.774874,-80.281936
223,PAST SALE,NaN,Condo/Co-op,1560 Jefferson Ave #7,Miami Beach,FL,33139.0,70000.0,NaN,1.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1560-Jef...,NaN,NaN,N,Y,25.788557,-80.137892


In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,NaN,Single Family Residential,851 SE 3rd Pl,Hialeah,FL,33010.0,515000.0,3.0,1.0,...,NaN,NaN,NaN,https://www.redfin.com/FL/Hialeah/851-SE-3rd-P...,NaN,NaN,N,Y,25.820451,-80.264125


In [9]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

2025-08-15    2
2025-08-18    1
2025-08-14    1
2025-08-13    1
Name: SOLD DATE, dtype: int64

In [14]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [15]:
# df2 = df2.drop(index=64)

In [16]:
df_top_ten = df2.head(10)

In [17]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
257,PAST SALE,2025-08-13,Condo/Co-op,18501 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,7700000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11843328,N,Y,25.947720,-80.120228
144,PAST SALE,2025-08-15,Single Family Residential,1260 99th St,Bay Harbor Islands,FL,33154.0,3022500.0,4.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11835741,N,Y,25.888807,-80.136107
98,PAST SALE,2025-08-14,Condo/Co-op,20 Calabria Ave #302,Coral Gables,FL,33134.0,585000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/20-Cala...,MARMLS,A11843932,N,Y,25.762772,-80.255534
90,PAST SALE,2025-08-18,Townhouse,6885 NW 169th St Unit 66D,Hialeah,FL,33015.0,325000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,MARMLS,A11819521,N,Y,25.926176,-80.313737
309,PAST SALE,2025-08-15,Condo/Co-op,600 Michigan Ave #1,Miami Beach,FL,33139.0,240000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/600-Mich...,MARMLS,A11845060,N,Y,25.775890,-80.138676


In [18]:
# df_top_ten = df2.head(10)

## Collect Agent Information


In [19]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [20]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [21]:
response_df = pd.DataFrame(response_list)

In [22]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [24]:
df_top_ten = merged_df

## Current Week's Values


In [25]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

5


In [26]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$2,374,500


In [27]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$971


In [28]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$11,872,500


In [29]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [30]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Input Previous Week Condo Sales Total (number units sold):
80
-----------
Input Previous Week Condo Average Sales Price:
1028539
-----------
Input Previous Week Condo Average PSF:
563
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
82283112.0


In [31]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,80,1028539,563,82283112.0


In [32]:
last_week['Last Week Total Dollar Volume'].iloc[0]

82283112.0

In [33]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0            5             2374500          971                     11872500.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story


In [34]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [35]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section


In [37]:
# df.at[index#,'col_name']

## Format Data


In [38]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale


In [39]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter


In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [41]:
pd.set_option('display.max_columns',None)

In [42]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-13,Condo/Co-op,18501 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,7700000.0,4.0,5.5,NORTH BISCAYNE BEACH AMEN,3252.0,NaN,2020.0,NaN,2368.0,5474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11843328,N,Y,25.94772,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roberto Costa,Opportunity Us Realty Corp,None,None,Denisse Levy,One Sotheby's International Realty,None,None,1,orange


In [43]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-13,Condo/Co-op,18501 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,7700000.0,4.0,5.5,NORTH BISCAYNE BEACH AMEN,3252.0,NaN,2020.0,NaN,2368.0,5474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11843328,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roberto Costa,Opportunity Us Realty Corp,None,None,Denisse Levy,One Sotheby's International Realty,None,None,1,orange
1,PAST SALE,2025-08-15,Single Family Residential,1260 99th St,Bay Harbor Islands,FL,33154.0,3022500.0,4.0,2.0,BAY HARBOR ISLAND,2093.0,10000.0,1954.0,NaN,1444.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11835741,N,Y,25.888807,-80.136107,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",2,blue
2,PAST SALE,2025-08-14,Condo/Co-op,20 Calabria Ave #302,Coral Gables,FL,33134.0,585000.0,2.0,2.0,VILLA CALABRIA CONDO,1202.0,NaN,2004.0,NaN,487.0,582.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/20-Cala...,MARMLS,A11843932,N,Y,25.762772,-80.255534,https://www.redfin.com/FL/Coral-Gables/20-Cala...,Michelle White,Michelle A White LLC,None,None,Reina Canosa Ramos,"Atlas Realty Company, Inc.",None,None,3,blue
3,PAST SALE,2025-08-18,Townhouse,6885 NW 169th St Unit 66D,Hialeah,FL,33015.0,325000.0,3.0,2.5,WOODS LANDING CONDO PH I,1590.0,NaN,1980.0,NaN,204.0,351.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,MARMLS,A11819521,N,Y,25.926176,-80.313737,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,Shani Maimoni,"Related ISG Realty, LLC.",None,None,Jennifer Gonzalez,Icaza Reyalty LLC,None,None,4,blue
4,PAST SALE,2025-08-15,Condo/Co-op,600 Michigan Ave #1,Miami Beach,FL,33139.0,240000.0,1.0,1.0,600 MICHIGAN CONDO,685.0,NaN,1950.0,NaN,350.0,438.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/600-Mich...,MARMLS,A11845060,N,Y,25.775890,-80.138676,https://www.redfin.com/FL/Miami-Beach/600-Mich...,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,5,blue


In [89]:
%store -r map_box_api_key

no stored variable or alias map_box_api_key


In [90]:
map_box_api_key = input("Enter Map Box API key")

In [91]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [92]:
m.save('index.html')

## Data snagger


In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [48]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-13,Condo/Co-op,18501 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,7700000.0,4.0,5.5,NORTH BISCAYNE BEACH AMEN,3252.0,NaN,2020.0,NaN,2368.0,5474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11843328,N,Y,25.94772,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roberto Costa,Opportunity Us Realty Corp,None,None,Denisse Levy,One Sotheby's International Realty,None,None,1,orange


In [49]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger


Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`


In [50]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_08182025


# CREATE TEMPLATE


In [51]:
muni_set = set(df_top_ten['CITY'])

In [52]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [53]:
df_top_ten.reset_index(inplace=True,drop=True)

In [54]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [55]:
top_sale

'NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave #3504 in Sunny Isles Beach'

In [56]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [57]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [58]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-13,Condo/Co-op,18501 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,7700000.0,4.0,5.5,NORTH BISCAYNE BEACH AMEN,3252.0,NaN,2020.0,NaN,2368.0,5474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11843328,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roberto Costa,Opportunity Us Realty Corp,None,None,Denisse Levy,One Sotheby's International Realty,None,None,1,orange,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
1,PAST SALE,2025-08-15,Single Family Residential,1260 99th St,Bay Harbor Islands,FL,33154.0,3022500.0,4.0,2.0,BAY HARBOR ISLAND,2093.0,10000.0,1954.0,NaN,1444.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11835741,N,Y,25.888807,-80.136107,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",2,blue,BAY HARBOR ISLAND at 1260 99th St in Bay Harbo...
2,PAST SALE,2025-08-14,Condo/Co-op,20 Calabria Ave #302,Coral Gables,FL,33134.0,585000.0,2.0,2.0,VILLA CALABRIA CONDO,1202.0,NaN,2004.0,NaN,487.0,582.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/20-Cala...,MARMLS,A11843932,N,Y,25.762772,-80.255534,https://www.redfin.com/FL/Coral-Gables/20-Cala...,Michelle White,Michelle A White LLC,None,None,Reina Canosa Ramos,"Atlas Realty Company, Inc.",None,None,3,blue,VILLA CALABRIA CONDO at 20 Calabria Ave #302 i...
3,PAST SALE,2025-08-18,Townhouse,6885 NW 169th St Unit 66D,Hialeah,FL,33015.0,325000.0,3.0,2.5,WOODS LANDING CONDO PH I,1590.0,NaN,1980.0,NaN,204.0,351.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,MARMLS,A11819521,N,Y,25.926176,-80.313737,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,Shani Maimoni,"Related ISG Realty, LLC.",None,None,Jennifer Gonzalez,Icaza Reyalty LLC,None,None,4,blue,WOODS LANDING CONDO PH I at 6885 NW 169th St U...
4,PAST SALE,2025-08-15,Condo/Co-op,600 Michigan Ave #1,Miami Beach,FL,33139.0,240000.0,1.0,1.0,600 MICHIGAN CONDO,685.0,NaN,1950.0,NaN,350.0,438.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/600-Mich...,MARMLS,A11845060,N,Y,25.775890,-80.138676,https://www.redfin.com/FL/Miami-Beach/600-Mich...,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,5,blue,600 MICHIGAN CONDO at 600 Michigan Ave #1 in M...


In [ ]:
max_index = 9
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[max_index,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[max_index,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: NORTH BISCAYNE BEACH AMEN closing tops miami-dade weekly condo sales 
DEK: Top 10 sales ranged from $240,000 to $7,700,000
FEATURED HED:
SEO HED: miami-dade County Weekly Condo Report 
SEO DESCRIPTION: miami-dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #miami-dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, miami-dade, weekly condo sales, Miami Beach, Coral Gables, Sunny Isles Beach, Hialeah, Bay Harbor Islands, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

miami-dade County’s NEWS PEG HERE.

Brokers closed 5 condo sales totaling $11,872,500 million from Aug. 12 to Aug. 18. The previous week, brokers closed 80 condo sales totaling $82,283,112.

Last week’s units sold for an average of $2,374,500, higher than the $1,028,539

In [ ]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [60]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [61]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [62]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-13,Condo/Co-op,18501 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,7700000.0,4.0,5.5,NORTH BISCAYNE BEACH AMEN,3252.0,NaN,2020.0,NaN,2368.0,5474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11843328,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roberto Costa,Opportunity Us Realty Corp,None,None,Denisse Levy,One Sotheby's International Realty,None,None,1,orange,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
1,PAST SALE,2025-08-15,Single Family Residential,1260 99th St,Bay Harbor Islands,FL,33154.0,3022500.0,4.0,2.0,BAY HARBOR ISLAND,2093.0,10000.0,1954.0,NaN,1444.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11835741,N,Y,25.888807,-80.136107,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",2,blue,BAY HARBOR ISLAND at 1260 99th St in Bay Harbo...
2,PAST SALE,2025-08-14,Condo/Co-op,20 Calabria Ave #302,Coral Gables,FL,33134.0,585000.0,2.0,2.0,VILLA CALABRIA CONDO,1202.0,NaN,2004.0,NaN,487.0,582.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/20-Cala...,MARMLS,A11843932,N,Y,25.762772,-80.255534,https://www.redfin.com/FL/Coral-Gables/20-Cala...,Michelle White,Michelle A White LLC,None,None,Reina Canosa Ramos,"Atlas Realty Company, Inc.",None,None,3,blue,VILLA CALABRIA CONDO at 20 Calabria Ave #302 i...
3,PAST SALE,2025-08-18,Townhouse,6885 NW 169th St Unit 66D,Hialeah,FL,33015.0,325000.0,3.0,2.5,WOODS LANDING CONDO PH I,1590.0,NaN,1980.0,NaN,204.0,351.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,MARMLS,A11819521,N,Y,25.926176,-80.313737,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,Shani Maimoni,"Related ISG Realty, LLC.",None,None,Jennifer Gonzalez,Icaza Reyalty LLC,None,None,4,blue,WOODS LANDING CONDO PH I at 6885 NW 169th St U...
4,PAST SALE,2025-08-15,Condo/Co-op,600 Michigan Ave #1,Miami Beach,FL,33139.0,240000.0,1.0,1.0,600 MICHIGAN CONDO,685.0,NaN,1950.0,NaN,350.0,438.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/600-Mich...,MARMLS,A11845060,N,Y,25.775890,-80.138676,https://www.redfin.com/FL/Miami-Beach/600-Mich...,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,5,blue,600 MICHIGAN CONDO at 600 Michigan Ave #1 in M...


In [96]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: NORTH BISCAYNE BEACH AMEN closing tops miami-dade weekly condo sales 
DEK: Top 10 sales ranged from $240,000 to $7,700,000
FEATURED HED:
SEO HED: miami-dade County Weekly Condo Report 
SEO DESCRIPTION: miami-dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #miami-dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, miami-dade, weekly condo sales, Miami Beach, Coral Gables, Sunny Isles Beach, Hialeah, Bay Harbor Islands, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

miami-dade County’s NEWS PEG HERE.

Brokers closed 5 condo sales totaling $11,872,500 million from Aug. 12 to Aug. 18. The previous week, brokers closed 80 condo sales totaling $82,283,112.

Last week’s units sold for an average of $2,374,500, higher than the $1,028,539

In [64]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Sunny-Isles-Beach/18501-Collins-Ave-33160/unit-3504/home/174905562


In [65]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Bay-Harbor-Islands/1260-99th-St-33154/home/42894325


In [66]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/600-Michigan-Ave-33139/unit-1/home/42804861


In [67]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hialeah/6885-NW-169th-St-33015/unit-66D/home/42963982


In [68]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/18501-Collins-Ave-33160/unit-3504/home/174905562


In [69]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Hialeah/6885-NW-169th-St-33015/unit-66D/home/42963982


In [70]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-13,Condo/Co-op,18501 Collins Ave #3504,Sunny Isles Beach,FL,33160.0,7700000.0,4.0,5.5,NORTH BISCAYNE BEACH AMEN,3252.0,NaN,2020.0,NaN,2368.0,5474.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11843328,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roberto Costa,Opportunity Us Realty Corp,None,None,Denisse Levy,One Sotheby's International Realty,None,None,1,orange,NORTH BISCAYNE BEACH AMEN at 18501 Collins Ave...
1,PAST SALE,2025-08-15,Single Family Residential,1260 99th St,Bay Harbor Islands,FL,33154.0,3022500.0,4.0,2.0,BAY HARBOR ISLAND,2093.0,10000.0,1954.0,NaN,1444.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11835741,N,Y,25.888807,-80.136107,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",Joel Lusky,"The Brokerage South Florida Real Estate, LLC.",Zalmy Shapira,"The Brokerage South Florida Real Estate, LLC.",2,blue,BAY HARBOR ISLAND at 1260 99th St in Bay Harbo...
2,PAST SALE,2025-08-14,Condo/Co-op,20 Calabria Ave #302,Coral Gables,FL,33134.0,585000.0,2.0,2.0,VILLA CALABRIA CONDO,1202.0,NaN,2004.0,NaN,487.0,582.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/20-Cala...,MARMLS,A11843932,N,Y,25.762772,-80.255534,https://www.redfin.com/FL/Coral-Gables/20-Cala...,Michelle White,Michelle A White LLC,None,None,Reina Canosa Ramos,"Atlas Realty Company, Inc.",None,None,3,blue,VILLA CALABRIA CONDO at 20 Calabria Ave #302 i...
3,PAST SALE,2025-08-18,Townhouse,6885 NW 169th St Unit 66D,Hialeah,FL,33015.0,325000.0,3.0,2.5,WOODS LANDING CONDO PH I,1590.0,NaN,1980.0,NaN,204.0,351.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,MARMLS,A11819521,N,Y,25.926176,-80.313737,https://www.redfin.com/FL/Hialeah/6885-NW-169t...,Shani Maimoni,"Related ISG Realty, LLC.",None,None,Jennifer Gonzalez,Icaza Reyalty LLC,None,None,4,blue,WOODS LANDING CONDO PH I at 6885 NW 169th St U...
4,PAST SALE,2025-08-15,Condo/Co-op,600 Michigan Ave #1,Miami Beach,FL,33139.0,240000.0,1.0,1.0,600 MICHIGAN CONDO,685.0,NaN,1950.0,NaN,350.0,438.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/600-Mich...,MARMLS,A11845060,N,Y,25.775890,-80.138676,https://www.redfin.com/FL/Miami-Beach/600-Mich...,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,Judy Gazzoni,"Related ISG Realty, LLC.",None,None,5,blue,600 MICHIGAN CONDO at 600 Michigan Ave #1 in M...


## Time on Market Calculator


In [71]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 5, 17) ## List (Earlier) date
date2 = datetime(2025, 8, 16) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

91


## Clean CSV for Datawrapper Chart


In [72]:
chart_df = df_top_ten

In [73]:
chart_df = chart_df.fillna(" ")

In [74]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [75]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [76]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [77]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [78]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [79]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [80]:
chart_df['PPSF'].astype(float)

0    2368.0
1    1444.0
2     487.0
3     204.0
4     350.0
Name: PPSF, dtype: float64

In [81]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [82]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [83]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-08-13,NORTH BISCAYNE BEACH AMEN,18501 Collins Ave #3504 in Sunny Isles Beach,"$7,700,000","3,252","$2,368",2020,Roberto Costa Opportunity Us Realty Corp,Denisse Levy One Sotheby's International Realt...
1,2025-08-15,BAY HARBOR ISLAND,1260 99th St in Bay Harbor Islands,"$3,022,500","2,093","$1,444",1954,Joel Lusky The Brokerage South Florida Real Es...,Joel Lusky The Brokerage South Florida Real Es...
2,2025-08-14,VILLA CALABRIA CONDO,20 Calabria Ave #302 in Coral Gables,"$585,000","1,202",$487,2004,Michelle White Michelle A White LLC,"Reina Canosa Ramos Atlas Realty Company, Inc."
3,2025-08-18,WOODS LANDING CONDO PH I,6885 NW 169th St Unit 66D in Hialeah,"$325,000","1,590",$204,1980,"Shani Maimoni Related ISG Realty, LLC.",Jennifer Gonzalez Icaza Reyalty LLC
4,2025-08-15,600 MICHIGAN CONDO,600 Michigan Ave #1 in Miami Beach,"$240,000",685,$350,1950,"Judy Gazzoni Related ISG Realty, LLC.","Judy Gazzoni Related ISG Realty, LLC."


In [84]:
csv_date_string = today.strftime("%m_%d_%Y")

In [85]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [86]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')